In [1]:
import os
import json
import openai
from dotenv import load_dotenv

# OpenAI APIs

- **Chat** (ChatGPT)
- Completion (Text, vorgänger von Chat)
- Embeddings (Text -> Vektor)
- Fine-Tuning
- Images 
- Audio

Links:
- [Dokumentation](https://platform.openai.com/docs/introduction)
- [API Reference](https://platform.openai.com/docs/api-reference)

## Chat

- Input: messages (Chatverlauf aus Nachrichten von: system, user, assistant)
- Output: assistant message oder function call
- Jeder API-Call besteht nur aus einer Frage-Antwort-Kombination, Kenntnis vom Chatverlauf muss selbst implementiert werden

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

Modelle:
- gpt-3.5-turbo
- gpt-4
- [viele weitere](https://platform.openai.com/docs/models)

Message Roles:
- `system`: Vorkonfiguration, wird in ChatGPT nicht als Chatnachricht angezeigt
- `user`: Nachrichten die vom Nutzer geschrieben wurden
- `assistant`: Antworten die ChatGPT gegeben hat
- `function`: Ergebnis einer Funktion

In [3]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ],
    temperature=0
)

In [4]:
dict(response)

{'id': 'chatcmpl-8QFrSs1cafqs1yN7OuuEjoYQWsK0L',
 'object': 'chat.completion',
 'created': 1701268190,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [<OpenAIObject at 0x1b0a150e210> JSON: {
    "index": 0,
    "message": {
      "role": "assistant",
      "content": "The 2020 World Series was played at Globe Life Field in Arlington, Texas."
    },
    "finish_reason": "stop"
  }],
 'usage': <OpenAIObject at 0x1b0a150e270> JSON: {
   "prompt_tokens": 53,
   "completion_tokens": 17,
   "total_tokens": 70
 }}

In [5]:
response["choices"][0]["message"]["content"]

'The 2020 World Series was played at Globe Life Field in Arlington, Texas.'

## Functions

- Schnittstelle der Modelle nach außen
- eigene Funktionen definieren
- dem Modell mitteilen, welche Funktionen zur Verfügung stehen 
- Modell entscheidet selbst, wann es Sinn ergibt eine Funktion aufzurufen
- API führt Funktionen nicht aus, sondern teilt nur mit, welche Funktion mit welchem Input ausgeführt werden soll

In [6]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [7]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",
        temperature=0
    )
    print("--------------- initial response ---------------")
    print(response)
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        print("--------------- second response ---------------")
        print(second_response)


run_conversation()

--------------- initial response ---------------
{
  "id": "chatcmpl-8QFrTwdW19cPF75S7uJPsNDmMIYAQ",
  "object": "chat.completion",
  "created": 1701268191,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}
--------------- second response ---------------
{
  "id": "chatcmpl-8QFrVZ5zmWE2CIaN3K2UWDkkdmPrk",
  "object": "chat.completion",
  "created": 1701268193,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston, MA is 72\u00b0F and sunny with windy conditions."
      },
      "finish_re